In [4]:
import pandas as pd;

In [59]:
cpc = pd.read_csv("./CPC.csv", encoding="ISO-8859-1", dtype=str)
cpc = cpc.set_index("CPC21code", drop=True)
cpc

,CPC21title
CPC21code,
0,"Agriculture, forestry and fishery products"
01,"Products of agriculture, horticulture and mark..."
011,Cereals
0111,Wheat
01111,"Wheat, seed"
...,...
98000,Domestic services
99,Services provided by extraterritorial organiza...
990,Services provided by extraterritorial organiza...


In [111]:
wanted = ["Element", "Item Code (CPC)", "Item", "Year", "Unit", "Value"]
primary_crops = pd.read_csv("./primary_crops.csv", dtype={"Item Code (CPC)":str}, usecols=wanted)
primary_crops

,Element,Item Code (CPC),Item,Year,Unit,Value
0,Area harvested,01371,"Almonds, in shell",1981,ha,120.0
1,Area harvested,01371,"Almonds, in shell",1982,ha,125.0
2,Area harvested,01371,"Almonds, in shell",1983,ha,125.0
3,Area harvested,01371,"Almonds, in shell",1984,ha,125.0
4,Area harvested,01371,"Almonds, in shell",1985,ha,129.0
...,...,...,...,...,...,...
7783,Production,0111,Wheat,2017,tonnes,95651.0
7784,Production,0111,Wheat,2018,tonnes,92210.0
7785,Production,0111,Wheat,2019,tonnes,100332.0
7786,Production,0111,Wheat,2020,tonnes,130000.0


In [244]:
def add_category(df):
  df["Category Code (CPC)"] = df["Item Code (CPC)"].str.slice(stop=3);
  keys = df["Category Code (CPC)"].isin(cpc.index)
  df.loc[keys, "Category"] = cpc.loc[df.loc[keys, "Category Code (CPC)"], "CPC21title"].array

In [112]:
add_category(primary_crops);

In [113]:
primary_crops

,Element,Item Code (CPC),Item,Year,Unit,Value,Category Code (CPC),Category
0,Area harvested,01371,"Almonds, in shell",1981,ha,120.0,013,Fruits and nuts
1,Area harvested,01371,"Almonds, in shell",1982,ha,125.0,013,Fruits and nuts
2,Area harvested,01371,"Almonds, in shell",1983,ha,125.0,013,Fruits and nuts
3,Area harvested,01371,"Almonds, in shell",1984,ha,125.0,013,Fruits and nuts
4,Area harvested,01371,"Almonds, in shell",1985,ha,129.0,013,Fruits and nuts
...,...,...,...,...,...,...,...,...
7783,Production,0111,Wheat,2017,tonnes,95651.0,011,Cereals
7784,Production,0111,Wheat,2018,tonnes,92210.0,011,Cereals
7785,Production,0111,Wheat,2019,tonnes,100332.0,011,Cereals
7786,Production,0111,Wheat,2020,tonnes,130000.0,011,Cereals


In [142]:
processed = primary_crops.pivot(
    index=["Item Code (CPC)", "Item", "Category Code (CPC)", "Category", "Year"],
    columns=["Unit","Element"],
    values="Value");
processed.columns = processed.columns.map('-'.join);
processed = processed.reset_index()


In [150]:
wanted = ["Year", "Unit", "Value"]
population = pd.read_csv("./population.csv", usecols=wanted, index_col="Year")
population["Population"] = population["Value"] * 1000;
population

,Unit,Value,Population
Year,,,
1950,1000 persons,4712.810,4712810.0
1951,1000 persons,4782.787,4782787.0
1952,1000 persons,4855.822,4855822.0
1953,1000 persons,4931.833,4931833.0
1954,1000 persons,5010.564,5010564.0
...,...,...,...
2017,1000 persons,30034.389,30034389.0
2018,1000 persons,30790.513,30790513.0
2019,1000 persons,31546.691,31546691.0


In [176]:
processed["tonnes-Production/cap"] = (processed["tonnes-Production"] /
                                      population.loc[processed["Year"], "Population"].values)


In [246]:
wanted = ["Partner Country Code (M49)", "Partner Countries", "Item Code (CPC)", "Item", "Year", "Unit", "Value", "Element"]
imports_exports = pd.read_csv("./imports_exports.csv", dtype={"Item Code (CPC)":str}, usecols=wanted)
add_category(imports_exports)
imports_exports = imports_exports.dropna();
imports_exports

,Partner Country Code (M49),Partner Countries,Element,Item Code (CPC),Item,Year,Unit,Value,Category Code (CPC),Category
0,4,Afghanistan,Import Quantity,01371,"Almonds, in shell",2013,tonnes,3.00,013,Fruits and nuts
1,4,Afghanistan,Import Quantity,01371,"Almonds, in shell",2015,tonnes,0.50,013,Fruits and nuts
2,4,Afghanistan,Import Quantity,01371,"Almonds, in shell",2018,tonnes,1.70,013,Fruits and nuts
3,4,Afghanistan,Import Quantity,01371,"Almonds, in shell",2019,tonnes,56.03,013,Fruits and nuts
4,4,Afghanistan,Import Value,01371,"Almonds, in shell",2013,1000 US$,6.00,013,Fruits and nuts
...,...,...,...,...,...,...,...,...,...,...
70925,716,Zimbabwe,Export Value,01970,Unmanufactured tobacco,2011,1000 US$,217.00,019,Forage products; fibre crops; plants used in p...
70928,716,Zimbabwe,Import Quantity,0111,Wheat,2009,tonnes,25010.00,011,Cereals
70929,716,Zimbabwe,Import Value,0111,Wheat,2009,1000 US$,5613.00,011,Cereals
70930,716,Zimbabwe,Import Quantity,22211,Whole milk powder,2005,tonnes,21.00,222,Other dairy products


In [259]:
imports_exports_primary = imports_exports[imports_exports.Category.isin(primary_crops.Category.unique())]

In [362]:
imports_exports_primary_pivot = imports_exports_primary.pivot(
    index=["Item Code (CPC)", "Item", "Category Code (CPC)", "Category", "Year", "Partner Countries", "Partner Country Code (M49)"],
    columns=["Unit","Element"],
    values="Value")
imports_exports_primary_pivot.columns = imports_exports_primary_pivot.columns.map('-'.join);
imports_exports_primary_pivot = imports_exports_primary_pivot.fillna(0);
imports_exports_primary_pivot

tonnes-Import Quantity   
Item Code (CPC) Item                                    Category Code (CPC) Category                                           Year Partner Countries    Partner Country Code (M49)                           
0111            Wheat                                   011                 Cereals                                            1998 Argentina            32                                        79390.00  \
                                                                                                                                    Australia            36                                       608486.00   
                                                                                                                                    Belgium-Luxembourg   58                                         9831.00   
                                                                                                                                    Brazil               76                                        50770.00   
                                                                                                                                    France               250                                       25839.00   
...                                                                                                                                                                                                     ...   
01990.01        Vegetable products, fresh or dry n.e.c. 019                 Forage products; fibre crops; plants used in pe... 2020 United Arab Emirates 784                                         167.49   
                                                                                                                               2021 Egypt                818                                          12.93   
                                                                                                                                    Oman                 512                                         596.27   
                                                                                                                                    Saudi Arabia         682                                           4.38   
                                                                                                                                    United Arab Emirates 784                                         374.18   

                                                                                                                                                                                     1000 US$-Import Value   
Item Code (CPC) Item                                    Category Code (CPC) Category                                           Year Partner Countries    Partner Country Code (M49)                          
0111            Wheat                                   011                 Cereals                                            1998 Argentina            32                                         5968.0  \
                                                                                                                                    Australia            36                                        47524.0   
                                                                                                                                    Belgium-Luxembourg   58                                          663.0   
                                                                                                                                    Brazil               76                                         3308.0   
                                                                                                                                    France               250                                        1727.0   
...                                            

In [363]:
imports_exports_primary_pivot["Import 1000 US$ per tonne"] = imports_exports_primary_pivot["tonnes-Import Quantity"] / imports_exports_primary_pivot["1000 US$-Import Value"];
imports_exports_primary_pivot["Export 1000 US$ per tonne"] = imports_exports_primary_pivot["tonnes-Export Quantity"] / imports_exports_primary_pivot["1000 US$-Export Value"];

In [364]:
imports_exports_primary_pivot["tonnes-Net Quantity"] = imports_exports_primary_pivot["tonnes-Import Quantity"] - imports_exports_primary_pivot["tonnes-Export Quantity"]

In [365]:
imports_exports_primary_pivot = imports_exports_primary_pivot.reset_index()

In [370]:
ie_pop = population.loc[imports_exports_primary_pivot["Year"], "Population"].values;
imports_exports_primary_pivot["tonnes-Import Quantity/cap"] = (imports_exports_primary_pivot["tonnes-Import Quantity"] / ie_pop)
imports_exports_primary_pivot["tonnes-Export Quantity/cap"] = (imports_exports_primary_pivot["tonnes-Export Quantity"] / ie_pop)
imports_exports_primary_pivot["1000 US$-Import Value/cap"] = (imports_exports_primary_pivot["1000 US$-Import Value"] / ie_pop)
imports_exports_primary_pivot["1000 US$-Export Value/cap"] = (imports_exports_primary_pivot["1000 US$-Export Value"] / ie_pop)

In [372]:
imports_exports_primary_pivot.to_csv("imports_exports_primary_processed.csv", na_rep='NULL')
processed.to_csv("primary_crops_processed.csv")

In [179]:
population.to_csv("population_processed.csv")